In [4]:
library(dplyr)   
library(caret)
library(tidyverse)
library(recipes)
library(pROC)
library(caTools)
library(UBL)

set.seed(1)

options(repr.plot.width = 20, repr.plot.height =10, repr.plot.res = 600)


load(file = "C:/Users/abonini/Desktop/Analysis/Sources/LIVERTRANSPLANT/Modello/df_finale.Rdata")

df <- df_finale

In [5]:
dim(df_finale)

[1] 1396   74

In [6]:
tmp <- subset(df, select = -c(deg_cat))           #hot encoding

#define one-hot encoding function
dummy <- dummyVars(" ~ .", data=tmp)

#perform one-hot encoding on data frame
final_df <- data.frame(predict(dummy, newdata=tmp))


final_df$deg_cat <- df$deg_cat


df <- final_df

## Data partition

In [7]:
#make this example reproducible
set.seed(1)

#use 70% of dataset as training set and 30% as test set
sample <- sample.split(df$deg_cat, SplitRatio = 0.7)
train  <- subset(df, sample == TRUE)
test   <- subset(df, sample == FALSE)



TestClasses <- test$deg_cat
TestData <- subset(test,select= -deg_cat)

In [8]:
dim(test)
dim(train)

[1] 419 175

[1] 977 175

In [9]:
train <- SmoteClassif(deg_cat ~ ., train, k = 27, dist = "HEOM")

#view distribution of response variable in new dataset
table(train$deg_cat)

TrainClasses <- train$deg_cat
TrainData <- subset(train,select= -deg_cat)


   fast NotFast 
    488     488 

## Model

In [10]:
ctrl <- trainControl(method = "repeatedcv", 
                 number = 10, 
                 savePredictions = "final",classProbs=TRUE,verboseIter=TRUE, search= "random") 

#summaryFunction = multiClassSummary (to keep all metrics)

In [11]:

model_svmPoly <- train(x=TrainData, y=TrainClasses, 
               method = "svmPoly", scale= FALSE) 

In [12]:

model_svmRadial <- train(x=TrainData, y=TrainClasses, 
               method = "svmRadial", scale= FALSE) 

Warning message in .local(x, ...):
"Variable(s) `' constant. Cannot scale data."


In [13]:

model_svmLinear <- train(x=TrainData, y=TrainClasses, 
               method = "svmLinear", scale= FALSE)

# Results

In [14]:
print(model_svmPoly)
plot(model_svmPoly,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Polynomial Kernel 

976 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 976, 976, 976, 976, 976, 976, ... 
Resampling results across tuning parameters:

  degree  scale  C     Accuracy   Kappa    
  1       0.001  0.25  0.6167833  0.2513749
  1       0.001  0.50  0.7059642  0.4153476
  1       0.001  1.00  0.7320881  0.4650689
  1       0.010  0.25  0.7369176  0.4745152
  1       0.010  0.50  0.7413936  0.4831578
  1       0.010  1.00  0.7391581  0.4785722
  1       0.100  0.25  0.7337228  0.4676560
  1       0.100  0.50  0.7295321  0.4591927
  1       0.100  1.00  0.7282295  0.4568143
  2       0.001  0.25  0.7081845  0.4198738
  2       0.001  0.50  0.7326366  0.4661658
  2       0.001  1.00  0.7368495  0.4744120
  2       0.010  0.25  0.7429787  0.4862168
  2       0.010  0.50  0.7431383  0.4865422
  2       0.010  1.00  0.7428157  0.4858191
  2       0.100  0.25  0.77

In [15]:
print(model_svmRadial)
plot(model_svmRadial,  cex= 1.5,  xlab = list( cex = 1.4), ylab = list( cex = 1.4), scales = list(x = list(cex=1.1),y=list(cex=1.1)))

Support Vector Machines with Radial Basis Function Kernel 

976 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 976, 976, 976, 976, 976, 976, ... 
Resampling results across tuning parameters:

  C     Accuracy   Kappa    
  0.25  0.7622511  0.5250954
  0.50  0.7667639  0.5341521
  1.00  0.7720222  0.5446478

Tuning parameter 'sigma' was held constant at a value of 0.01862808
Accuracy was used to select the optimal model using the largest value.
The final values used for the model were sigma = 0.01862808 and C = 1.


In [16]:
print(model_svmLinear)
plot(model_svmLinear)

Support Vector Machines with Linear Kernel 

976 samples
174 predictors
  2 classes: 'fast', 'NotFast' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 976, 976, 976, 976, 976, 976, ... 
Resampling results:

  Accuracy   Kappa    
  0.7165738  0.4331121

Tuning parameter 'C' was held constant at a value of 1


ERROR: Error in plot.train(model_svmLinear): There are no tuning parameters with more than 1 value.


In [17]:
# collect resamples
results <- resamples(list(Poly=model_svmPoly,  Radial=model_svmRadial, Linear=model_svmLinear))
# summarize the distributions
summary(results)
# boxplots of results
bwplot(results)
# dot plots of results
dotplot(results)


Call:
summary.resamples(object = results)

Models: Poly, Radial, Linear 
Number of resamples: 25 

Accuracy 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.7287671 0.7669377 0.7872928 0.7835828 0.7982955 0.8236994    0
Radial 0.7081081 0.7640751 0.7746479 0.7720222 0.7924528 0.8074866    0
Linear 0.6840659 0.6976744 0.7138965 0.7165738 0.7303030 0.7666667    0

Kappa 
            Min.   1st Qu.    Median      Mean   3rd Qu.      Max. NA's
Poly   0.4573265 0.5342960 0.5752339 0.5670248 0.5917941 0.6450267    0
Radial 0.4232268 0.5275899 0.5518376 0.5446478 0.5844377 0.6149733    0
Linear 0.3678647 0.3956553 0.4290519 0.4331121 0.4589367 0.5350554    0


# Prediction

In [18]:
pred_svmPoly <- predict(model_svmPoly,TestData)
pred_svmPoly.prob <- predict(model_svmPoly,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmPoly) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast      93      53
   NotFast   73     200
                                          
               Accuracy : 0.6993          
                 95% CI : (0.6529, 0.7428)
    No Information Rate : 0.6038          
    P-Value [Acc > NIR] : 3.032e-05       
                                          
                  Kappa : 0.3582          
                                          
 Mcnemar's Test P-Value : 0.09052         
                                          
            Sensitivity : 0.5602          
            Specificity : 0.7905          
         Pos Pred Value : 0.6370          
         Neg Pred Value : 0.7326          
             Prevalence : 0.3962          
         Detection Rate : 0.2220          
   Detection Prevalence : 0.3484          
      Balanced Accuracy : 0.6754          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5602410            0.7905138            0.6369863 
      Neg Pred Value            Precision               Recall 
           0.7326007            0.6369863            0.5602410 
                  F1           Prevalence       Detection Rate 
           0.5961538            0.3961814            0.2219570 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.6753774

In [19]:
pred_svmRadial <- predict(model_svmRadial,TestData)
pred_svmRadial.prob <- predict(model_svmRadial,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmRadial) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     103      43
   NotFast   76     197
                                          
               Accuracy : 0.716           
                 95% CI : (0.6702, 0.7587)
    No Information Rate : 0.5728          
    P-Value [Acc > NIR] : 9.631e-10       
                                          
                  Kappa : 0.4058          
                                          
 Mcnemar's Test P-Value : 0.003352        
                                          
            Sensitivity : 0.5754          
            Specificity : 0.8208          
         Pos Pred Value : 0.7055          
         Neg Pred Value : 0.7216          
             Prevalence : 0.4272          
         Detection Rate : 0.2458          
   Detection Prevalence : 0.3484          
      Balanced Accuracy : 0.6981          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5754190            0.8208333            0.7054795 
      Neg Pred Value            Precision               Recall 
           0.7216117            0.7054795            0.5754190 
                  F1           Prevalence       Detection Rate 
           0.6338462            0.4272076            0.2458234 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.6981262

In [20]:
pred_svmLinear <- predict(model_svmLinear,TestData)
pred_svmLinear.prob <- predict(model_svmLinear,TestData, type="prob")


result <- confusionMatrix(TestClasses,  pred_svmLinear) 
result
result$byClass

Warning message in method$prob(modelFit = modelFit, newdata = newdata, submodels = param):
"kernlab class probability calculations failed; returning NAs"


Confusion Matrix and Statistics

          Reference
Prediction fast NotFast
   fast     106      40
   NotFast   79     194
                                          
               Accuracy : 0.716           
                 95% CI : (0.6702, 0.7587)
    No Information Rate : 0.5585          
    P-Value [Acc > NIR] : 2.309e-11       
                                          
                  Kappa : 0.4111          
                                          
 Mcnemar's Test P-Value : 0.000495        
                                          
            Sensitivity : 0.5730          
            Specificity : 0.8291          
         Pos Pred Value : 0.7260          
         Neg Pred Value : 0.7106          
             Prevalence : 0.4415          
         Detection Rate : 0.2530          
   Detection Prevalence : 0.3484          
      Balanced Accuracy : 0.7010          
                                          
       'Positive' Class : fast            
               

Sensitivity          Specificity       Pos Pred Value 
           0.5729730            0.8290598            0.7260274 
      Neg Pred Value            Precision               Recall 
           0.7106227            0.7260274            0.5729730 
                  F1           Prevalence       Detection Rate 
           0.6404834            0.4415274            0.2529833 
Detection Prevalence    Balanced Accuracy 
           0.3484487            0.7010164

In [21]:
roc_obj <- roc(TestClasses, as.numeric(pred_svmRadial))
plot(roc_obj, main="Curva ROC")


Setting levels: control = fast, case = NotFast

Setting direction: controls < cases



In [22]:
roc_obj


Call:
roc.default(response = TestClasses, predictor = as.numeric(pred_svmRadial))

Data: as.numeric(pred_svmRadial) in 146 controls (TestClasses fast) < 273 cases (TestClasses NotFast).
Area under the curve: 0.7135